In [2]:
from google.colab import drive
import os

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00


In [4]:
!cp /content/drive/MyDrive/gpt2-noticias/paper/Utils/* /content

In [5]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip -qo v0.9.2.zip
!cd fastText-0.9.2 && pip install . -qU

--2023-11-26 00:31:11--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/refs/tags/v0.9.2 [following]
--2023-11-26 00:31:11--  https://codeload.github.com/facebookresearch/fastText/zip/refs/tags/v0.9.2
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.2.zip’

v0.9.2.zip              [  <=>               ]   4.17M  19.3MB/s    in 0.2s    

2023-11-26 00:31:11 (19.3 MB/s) - ‘v0.9.2.zip’ saved [4369852]

  Preparing metadata (setup.py) ... done


In [6]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
from sklearn.model_selection import train_test_split
import WordEmbeddings as we
import LoadData as load
import pandas as pd
import FileUtil as f

In [8]:
def concatLists(l1,l2):
    l3 = []
    for i,d in enumerate(l1):
        l3.append('__label__'+str(l2[i]) + ' ' +str(l1[i]))
    return l3


In [9]:
threshold = 0.9995
max_iter = 15
train_only = True #considera a amostra recem-classificada como train e nao faz train/test split

dataset = "/content/drive/MyDrive/gpt2-noticias/dataset_final.txt"

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="semi_supervised_fasttext_sa",

    name=f"thr {threshold} - (train_only {train_only})",
    # track hyperparameters and run metadata
    config={
      'threshold' : threshold,
      'max_iter' : max_iter,
      'train_only' : train_only,
      'dataset' :dataset
    }
)

config = wandb.config

dataSource = 'news'
random_state = 42

verbose = True

training_path = './trainFT.txt'
test_path = './testFT.txt'

wandb: Currently logged in as: mmakita. Use `wandb login --relogin` to force relogin


In [10]:
df1 = pd.DataFrame({'text': open("/content/drive/MyDrive/gpt2-noticias/dataset_pos.txt", "r").readlines()})
df1['sentiment'] = int(1)
df0 = pd.DataFrame({'text':open("/content/drive/MyDrive/gpt2-noticias/dataset_neg.txt", "r").readlines()})
df0['sentiment'] = int(0)

df_orig = pd.concat([df1,df0])

In [11]:
from numpy import NaN
unclassified_data = open(config.dataset, "r").readlines()
df = pd.DataFrame({'text':unclassified_data,'sentiment':NaN})
df

,text,sentiment
0,...,NaN
1,...,NaN
2,...,NaN
3,...,NaN
4,...,NaN
...,...,...
229634,...,NaN
229635,...,NaN
229636,...,NaN
229637,...,NaN


In [12]:
def calc_metrics():
  data = df_orig['text'].tolist()
  labels = df_orig['sentiment'].tolist()

  train_x, test_x, train_y, test_y = train_test_split(data,labels,stratify=labels,test_size=0.2, random_state = random_state)
  print(f'Testando em {len(test_x)} amostras')

  pred = []
  for sample in test_x:
    pred.append(int(model.predict(sample.strip())[0][0][-1]))

  from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

  print('Acc: '+str(accuracy_score(test_y, pred))+'\n'+
        'Precision: '+ str(precision_score(test_y, pred))+'\n'+
        'Recall: '+str(recall_score(test_y, pred))+'\n'+
        'F1: '+str(f1_score(test_y, pred))
        )

  wandb.log({
      'Acc': accuracy_score(test_y, pred),
      'Precision': precision_score(test_y, pred),
      'Recall': recall_score(test_y, pred),
      'F1':f1_score(test_y, pred)
  })

  model.save_model('./FastText_'+dataSource+'.mod')

In [13]:
data = df_orig['text'].tolist()
labels = df_orig['sentiment'].tolist()

train_x, test_x, train_y, test_y = train_test_split(data,labels,stratify=labels,test_size=0.2, random_state = random_state)

In [14]:
new_samples = []
new_labels = []

for iter in range(config.max_iter):
  if verbose:
    print(f'=== Iter {iter} ===')

  #SPLIT
  if len(new_samples) > 0 and config.train_only == False:
    new_train_x, new_test_x, new_train_y, new_test_y = train_test_split(new_samples,new_labels,stratify=new_labels,test_size=0.2, random_state = random_state)
    train_x = train_x + new_train_x
    test_x = test_x + new_test_x
    train_y = train_y + new_train_y
    test_y = test_y + new_test_y
  elif len(new_samples) > 0:
    train_x = train_x + new_samples
    train_y = train_y + new_labels

  if verbose:
    print(f' Treinando {len(train_x+test_x)} amostras (train: {len(train_x), len(train_y)}, test: {len(test_x), len(test_y)})')

  wandb.log({'Train_size': len(train_x)})

  f.saveListToFile(training_path,concatLists(train_x,train_y)) #train
  f.saveListToFile(test_path, concatLists(test_x,test_y)) #test

  #TRAIN
  model = we.fastText_Classification(training_path, test_path)

  unclassified_data_old = unclassified_data.copy()
  unclassified_data = []
  new_samples = []
  new_labels = []

  for i, d in enumerate(unclassified_data_old):

    p = model.predict(d.strip())

    if verbose and i%5000 == 0:
      print(f'Amostra {i}/{len(unclassified_data_old)}: {d.strip()} Pred:' ,p[0][0][-1],' Score: ', round(p[1][0],2))

    if p[1][0] >= config.threshold:
      new_samples.append(d)
      new_labels.append(int(p[0][0][-1]))
    else:
      unclassified_data.append(d)

  print(f'Dataset tera {len(new_samples), len(new_labels)} novas amostras. Restam {len(unclassified_data)} amostras.')

  ##TEST
  calc_metrics()



  print(f'Dataset tera {len(new_samples), len(new_labels)} novas amostras. Restam {len(unclassified_data)} amostras.')

  if len(unclassified_data) < 0.1*len(data) or len(new_samples) < 0.01*len(train_x+test_x) or iter > config.max_iter:
    break


=== Iter 0 ===
 Treinando 648 amostras (train: (518, 518), test: (130, 130))
Amostra 0/229639: Ações da Vale sobem 8,3% após notícia sobre resultados\n Pred: 1  Score:  1.0
Amostra 500/229639: Polícia apreende arsenal de R$ 15 mil no interior de SP\n Pred: 1  Score:  0.92
Amostra 1000/229639: Ibovespa sobe mais de 2% e tem maior nível desde março de 2008\n Pred: 0  Score:  0.8
Amostra 1500/229639: Preocupação com Brasil piorou e dólar subiu a R$ 1,596\n Pred: 0  Score:  0.9
Amostra 2000/229639: Motociclista de 24 anos morre após bater em caminhão em rodovia de Itabuna\n Pred: 1  Score:  0.66
Amostra 2500/229639: Polícia faz busca por suspeitos de assaltar bancos em São Félix dos Campos, MG\n Pred: 0  Score:  0.83
Amostra 3000/229639: Bancos brasileiros têm taxas reduzidas em abril, aponta Procon\n Pred: 0  Score:  0.98
Amostra 3500/229639: BNDES vê aumento dos juros básicos em 9,5% a 11% em 2015\n Pred: 0  Score:  0.5
Amostra 4000/229639: Dólar fecha abaixo de R$ 3,70 pela primeira vez

In [15]:
len(new_samples), len(new_labels)+518

(1760, 2278)

In [16]:
wandb.finish()

Acc,█▇▆▆▆▆▅▃▂▂▁
F1,█▇▆▆▆▆▄▃▂▂▁
Precision,▁▂▄▆███▇▇▇▇
Recall,█▇▆▅▅▅▃▂▂▂▁
Train_size,▁▁▃▄▆▇▇████
Acc,0.81538
F1,0.76471
Precision,0.90698
Recall,0.66102
Train_size,208022
